## Apply MediaPipe on video files/save video with landmarks

This notebook applies MediaPipe Hollistic on a given video file. It detects face, pose and hand landmarks. These landmarks are tracked over time and added to the video file. 
The video with landmarks added is saved as mp4 file at the end of the notebook.

### Import of libraries

In [29]:
import cv2
import mediapipe as mp
import numpy as np

### Helper Functions

In [30]:
mp_holistic = mp.solutions.holistic # holistic model
mp_drawing = mp.solutions.drawing_utils # drawing utilities

# function to detect MP Holistic landmarks from an image, e.g. frames of your camera feed
def mediapipe_detection(image, model): 
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # color conversion BGR to RGB
    image.flags.writeable = False                   # image no longer writeable
    results = model.process(image)                  # make prediction
    image.flags.writeable = True                    # image is writeable again
    #image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # color conversion back to original
    return image, results

In [31]:
# function to draw landmarks points and connecting lines on top of an image, e.g. on top of your camera feed
def draw_styled_landmarks(image, results): 
    # draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                              mp_drawing.DrawingSpec(color=(80,22,10), thickness=1, circle_radius=2), 
                              mp_drawing.DrawingSpec(color=(224,208,64), thickness=2, circle_radius=2))
    #draw pose connections
    #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
    #                          mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
    #                          mp_drawing.DrawingSpec(color=(224,208,64), thickness=2, circle_radius=2)) 
    #draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                              mp_drawing.DrawingSpec(color=(255,0,0), thickness=4, circle_radius=3), 
                              mp_drawing.DrawingSpec(color=(255,0,255), thickness=6, circle_radius=2))
    # draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                              mp_drawing.DrawingSpec(color=(255,0,0), thickness=4, circle_radius=3), 
                              mp_drawing.DrawingSpec(color=(255,0,255), thickness=6, circle_radius=2))

In [ ]:
#defining input directory
INPUT = '../data/'

## Loading and processing of video file

In [32]:
# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture(f'{INPUT}20230428_112427.mp4')
# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
#video will be saved as mp4 file
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 25.0, (frame_width,frame_height))

# Read until video is completed
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic: 
  while(cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
      # Display the resulting frame
      # make detections 
      image, results = mediapipe_detection(frame, holistic)
      #draw_landmarks(image, results)
      draw_styled_landmarks(image, results)
      out.write(frame)
      cv2.imshow('Frame',frame)
      # Press Q on keyboard to  exit
      if cv2.waitKey(25) & 0xFF == ord('q'):
        break
    # Break the loop
    else: 
      break
    
# When everything done, release the video capture object
cap.release()
out.release()
# Closes all the frames
cv2.destroyAllWindows()
cv2.waitKey(1) # some workaround to fix the bug, that window doesn't close


-1

: 